In [ ]:
import torch
import numpy as np
level=4
coords = np.arange(2**level,dtype=np.int16)
points = np.array(np.meshgrid(coords, coords, coords, indexing='xy'))
points = points.transpose(3,2,1,0).reshape(-1, 3)
points = torch.from_numpy(points).short().cuda()

from kaolin import _C

def points_to_morton(points):
    r"""Convert (quantized) 3D points to morton codes.

    Args:
        points (torch.ShortTensor):
            Quantized 3D points. This is not exactly like SPC points hierarchies
            as this is only the data for a specific level,
            of shape :math:`(\text{num_points}, 3)`.

    Returns:
        (torch.LongTensor):
            The morton code of the points,
            of shape :math:`(\text{num_points})`

    Examples:
        >>> inputs = torch.tensor([
        ...     [0, 0, 0],
        ...     [0, 0, 1],
        ...     [0, 0, 2],
        ...     [0, 0, 3],
        ...     [0, 1, 0]], device='cuda', dtype=torch.int16)
        >>> points_to_morton(inputs)
        tensor([0, 1, 8, 9, 2], device='cuda:0')
    """
    shape = list(points.shape)[:-1]
    points = points.reshape(-1, 3)
    return _C.ops.spc.points_to_morton_cuda(points.contiguous()).reshape(*shape)

def morton_to_points(morton):
    r"""Convert morton codes to points.

    Args:
        morton (torch.LongTensor): The morton codes of quantized 3D points,
                                   of shape :math:`(\text{num_points})`.

    Returns:
        (torch.ShortInt):
            The points quantized coordinates,
            of shape :math:`(\text{num_points}, 3)`.

    Examples:
        >>> inputs = torch.tensor([0, 1, 8, 9, 2], device='cuda')
        >>> morton_to_points(inputs)
        tensor([[0, 0, 0],
                [0, 0, 1],
                [0, 0, 2],
                [0, 0, 3],
                [0, 1, 0]], device='cuda:0', dtype=torch.int16)
    """
    shape = list(morton.shape)
    shape.append(3)
    morton = morton.reshape(-1)
    return _C.ops.spc.morton_to_points_cuda(morton.contiguous()).reshape(*shape)

points = torch.unique(points.contiguous(), dim=0).contiguous()
points = torch.sort(points_to_morton(points).contiguous())[0]
points = morton_to_points(points.contiguous())
a=_C.ops.spc.points_to_octree(points.contiguous(), level)
a.shape
a
np.sum(np.power(np.power(2,np.arange(level)),3))